In [2]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
from nltk.stem import PorterStemmer
import numpy as np
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

2024-02-15 10:51:05.870258: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 10:51:05.870407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 10:51:06.067982: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:
val_data = pd.read_csv('/kaggle/input/emotion-dataset/validation.csv')
train_data = pd.read_csv('/kaggle/input/emotion-dataset/training.csv')
test_data = pd.read_csv('/kaggle/input/emotion-dataset/test.csv')
half_test_data = test_data.iloc[1000:]
test_data = test_data.iloc[:1000]
val_data = pd.concat([val_data, half_test_data], axis=0)
labels_dict = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
test_data['label_name'] = test_data['label'].map(labels_dict)
test_data.groupby(["label_name","label"]).size()
all_list = train_data['text'].tolist() + test_data['text'].tolist() + val_data['text'].tolist()
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(all_list)
word_index1 = tokenizer1.word_index
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in word_index1.keys()]
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(stemmed_words)
word_index2 = tokenizer2.word_index

In [42]:
model = load_model('/kaggle/input/sentiment-analysis/sentiment.h5')

In [51]:
def predict_data(data):
    new_data = []
    test_split = data.split()
    stemmed_words2 = [stemmer.stem(word) for word in test_split]
    token_list= tokenizer2.texts_to_sequences([stemmed_words2])[0]
    new_data.append([token_list]) 
    X = [row[0] for row in new_data]
    for j in range(66-len(X[0])):
        X[0].append(0)
    X = np.array(X)
    arr = model.predict(X)
    max_index = np.argmax(arr)
    binary_arr = np.zeros_like(arr)
    binary_arr[0, max_index] = 1
    if max_index<6:
        label = labels_dict[max_index]
        return label
    return "NA"

In [54]:
data = "i felt anger when at the end of a telephone call"
new_data = predict_data(data)

1/1 [==============================] - 0s 53ms/step


In [55]:
new_data

'anger'